## Plan

- set up standard operators
- function to make tensor products with trivial operators
- function to check linear independence (rank=#operators iff rows linearly dependent)
- function to build closed lie algebras (with cut off)
- function to extend lie algebra with commutations of another lie algebra
- create tests from examples in paper

##### Problems

- original Ops in complete_algebra need to be linearly independent: check "complete_algebra([X, X], 5)"
- cannot determine how an operator is linearly dependent on operators in algebra (use qr decomp: cols of r give dependencies)
- final Lie algebra is list of huge matrices, not their tensor decomposition (way to simplify down from linear dependencies?)

In [93]:
import numpy as np

In [94]:
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [103]:
def tensor(W: np.ndarray, dim: int, start: int, end: int = 0):
    """
    Return tensor product of dim operators acting with W on qubits [start, end] and trivially elsewhere.
    
    Parameters
    ----------
    W : np.ndarray
        Operator acting on quibits [start, end]
    dim : int
        Number of qubits in system
    start : int
        First non-trivial operator
    end : int
        Final non-trivial operator

    Returns
    -------
    T : np.ndarray
        Tensor product acting with W on qubits [start, end] and trivially elsewhere
    """
    if not end:
        end = start

    op = [I for i in range(dim)]

    for i in range(start, end+1):
        op[i] = W

    T = op[0]
    for i in range(1, dim):
        T = np.kron(T, op[i])

    return T

In [96]:
def lin_ind(Ops: list):
    """
    Determines whether a list of operators are linearly independent.
    
    Parameters
    ----------
    Ops : list
        List of operators with equal dimension

    Returns
    -------
    ind : bool
        List of operators are linearly independent, true or false
    """
    # matrix such that each column corresponds to one operator
    L = np.array([np.squeeze(op.reshape((1, -1))) for op in Ops]).T

    rank = np.linalg.matrix_rank(L)
    ind = (rank==len(Ops))
    return ind

In [149]:
class MaxOperators(Exception):
    pass

class LinearIndependence(Exception):
    pass

In [98]:
def complete_algebra_inner(Ops: list, start: int):
    """Find all linearly independent operators from commutations of operators in Ops[0:len(Ops)] with operators in Ops[start:len(Ops)]."""
    new_Ops = Ops
    for i in range(len(Ops)):
        for j in range(max(i, start), len(Ops)):
            new_op = new_Ops[i]@new_Ops[j] - new_Ops[j]@new_Ops[i]
            new_Ops.append(new_op)
            if not lin_ind(new_Ops):
                new_Ops.pop()
    return new_Ops

In [150]:
def complete_algebra(Ops: list, max: int, start: int = 0):
    """
    Find closed Lie algebra given initial set of operators.

    Parameters
    ----------
    Ops : list
        List of initial operators in Lie algebra
    max : int
        Cut off after max number of operators in Lie algebra found
    start : int
        Operator index to start verifying closedness, ie every commutation with operators before start already accounted  

    Returns
    -------
    new_Ops : list
        List of operators in completed Lie algebra
    """
    if not lin_ind(Ops):
        raise LinearIndependence("Given operators are not linearly independent.")

    old_Ops = Ops

    while True:
        # stop if maximum operators in algebra reached
        if len(old_Ops) > max:
            raise MaxOperators(f"Maximum of {max} operators in algebra reached.")
        
        # find new set of linearly independent operators to extend old_Ops
        new_Ops = complete_algebra_inner(old_Ops, start)

        # number of new operators added
        added_ops = len(new_Ops) - len(old_Ops)

        # if no new operators found, algebra is complete
        if added_ops == 0:
            return new_Ops
        else:
            start = len(old_Ops)
            old_Ops = new_Ops

In [121]:
def find_algebra(Lie_0: list, Lie_1: list, max: int):
    """
    Extend Lie algebra Lie_0 to include commutations with operators in Lie_1.

    Parameters
    ----------
    Lie_0 : list
        List of operators forming a Lie algebra
    Lie_1 : list
        List of operators to extend Lie_0 with commutations of
    max : int
        Cut off after max number of operators in Lie algebra found

    Returns
    -------
    Lie_alg : list
        List of operators in extended Lie algebra
    """
    Ops = Lie_0
    # append every commutation that is linearly independent
    for i in range(len(Lie_0)):
        for j in range(len(Lie_1)):
            new_op = Lie_0[i]@Lie_1[j] - Lie_1[j]@Lie_0[i]
            Ops.append(new_op)
            if not lin_ind(Ops):
                Ops.pop()
    # complete the algebra, starting from added operators
    Lie_alg = complete_algebra(Ops, max, start=len(Lie_0)-1)
    return Lie_alg
            

In [151]:
complete_algebra([X, I, Z, A, B], 10)

LinearIndependence: Given operators are not linearly independent.

In [146]:
lin_ind([X,I,Z,A,B])

False

In [148]:
find_algebra([X, I], [Z, A, B], 10)

[array([[0, 1],
        [1, 0]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[ 0, -2],
        [ 2,  0]]),
 array([[ 1, -1],
        [ 1, -1]])]

In [125]:
A = np.array([[1, 0], [1, 0]])
B = np.array([[2, 0], [1, 1]])

In [102]:
lin_ind([A, I, B])

False